In [21]:
!pip install langchain
!pip install openai
!pip install pandas
!pip install transformers
!pip install PyPDF2
!pip install python-dotenv
!pip install faiss-cpu

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
     ---- -------------------------------- 30.7/232.6 kB 445.2 kB/s eta 0:00:01
     ----------------- ------------------ 112.6/232.6 kB 939.4 kB/s eta 0:00:01
     -------------------------------------- 232.6/232.6 kB 1.4 MB/s eta 0:00:00


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [19]:
import os
import pandas as pd
from transformers import GPT2TokenizerFast

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain

from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [24]:
import PyPDF2

def pdf_to_text(pdf_path, txt_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                txt_file.write(page.extract_text())

pdf_to_text(
    pdf_path='big_data_for_dummies.pdf',
    txt_path='big_data_for_dummies.txt'
)

In [25]:
text = ''
with open('big_data_for_dummies.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [27]:
text = ''
with open('transformers.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [28]:
def count_tokens(text: str) -> int:
    tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

In [29]:
from dotenv import dotenv_values

config = dotenv_values('.env')
OPEN_API_KEY = config.get('OPEN_API_KEY')

embeddings = OpenAIEmbeddings(openai_api_key=OPEN_API_KEY, model='text-embedding-ada-002')

db = FAISS.from_documents(chunks, embeddings)

In [33]:
query = 'What is transformers?'
docs = db.similarity_search(query)

In [34]:
docs

[Document(page_content='LLMs, or Large Language Models, are the key component behind text generation. In a nutshell, they consist of large pretrained transformer models trained to predict the next word (or, more precisely, token) given some input text. Since they predict one token at a time, you need to do something more elaborate to generate new sentences other than just calling the model — you need to do autoregressive generation.\n\nAutoregressive generation is the inference-time procedure of iteratively calling a model with its own generated outputs, given a few initial inputs. In 🤗 Transformers, this is handled by the generate() method, which is available to all models with generative capabilities.\n\nThis tutorial will show you how to:\n\nGenerate text with an LLM\nAvoid common pitfalls\nNext steps to help you get the most out of your LLM\nBefore you begin, make sure you have all the necessary libraries installed:\n\nCopied\npip install transformers bitsandbytes>=0.39.0 -q\nGener

In [35]:
chain = load_qa_chain(OpenAI(openai_api_key=OPEN_API_KEY, temperature=0.1), chain_type='stuff')
chain.run(input_documents=docs, question=query)

'\n\nTransformers are large pretrained transformer models used for text generation. They are trained to predict the next word or token given some input text and are used for autoregressive generation. This process involves iteratively calling the model with its own generated outputs to generate new sentences. Transformers are resource-intensive and should be executed on a GPU for optimal results.'